### IMPORTING DATASETS

In [3]:
import os
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

In [4]:
os.getcwd()

os.chdir("C:\\Users\\DSAMIT\\TEST\\Recommendation_Engine_Module")

df_movies = pd.read_csv("movies_clean.csv")

df_ratings = pd.read_csv("reviews_clean.csv")

df_movies = df_movies.iloc[:,1:]
df_ratings=df_ratings.iloc[:,1:]

In [21]:
df_mr

,user_id,movie_id,rating,timestamp,date,movie,genre
0,1,68646,10,1381620027,2013-10-12 23:20:27,The Godfather (1972),Crime|Drama
1,1,113277,10,1379466669,2013-09-18 01:11:09,Heat (1995),Action|Crime|Drama
2,2,422720,8,1412178746,2014-10-01 15:52:26,Marie Antoinette (2006),Biography|Drama|History
3,2,454876,8,1394818630,2014-03-14 17:37:10,Life of Pi (2012),Adventure|Drama|Fantasy
4,2,790636,7,1389963947,2014-01-17 13:05:47,Dallas Buyers Club (2013),Biography|Drama
5,2,816711,8,1379963769,2013-09-23 19:16:09,World War Z (2013),Action|Adventure|Horror
6,2,1091191,7,1391173869,2014-01-31 13:11:09,Lone Survivor (2013),Action|Biography|Drama
7,2,1103275,7,1408192129,2014-08-16 12:28:49,Two Lovers (2008),Drama|Romance
8,2,1322269,7,1391529691,2014-02-04 16:01:31,August: Osage County (2013),Drama
9,2,1390411,8,1451374513,2015-12-29 07:35:13,In the Heart of the Sea (2015),Action|Adventure|Biography


In [5]:
df_mr = pd.merge(df_ratings,df_movies.loc[:,["movie_id","movie","genre"]],how = "left",on="movie_id")

In [6]:
df_new = df_mr.loc[:,["user_id","rating","movie"]]

In [7]:
df_new["user_id"].nunique()

55789

In [8]:
df_new100=df_new.loc[df_new["user_id"]<=100,:] #taking only 100 users

In [9]:
df_new100_R=df_new100.pivot(index = "user_id",columns = "movie",values = "rating").fillna(0)

df_new100_mat = df_new100_R.as_matrix()##converting into matrix for svd

C:\Users\DSAMIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
def apply_svd(k):
    U, sigma, Vt=svds(df_new100_mat,k)
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    return all_user_predicted_ratings

In [11]:
df_svd = pd.DataFrame(apply_svd(25),columns = df_new100_R.columns)

In [12]:
user_id=pd.Series(list(range(1,101,1)))

df_svd_new = pd.concat([user_id,df_svd],axis=1).rename(columns = {0:"user_id"})

In [13]:
df_svd_new.head()

,user_id,12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),2 Guns (2013),2001: A Space Odyssey (1968),20th Century Women (2016),21 Jump Street (2012),22 Jump Street (2014),3 Idiots (2009),...,You're Next (2011),You're Not You (2014),Young Adult (2011),Zamani barayé masti asbha (2000),Zendegi-e Khosousi-e Agha Va Khanom-e Mim (2012),Zero Dark Thirty (2012),Zodiac (2007),Zoolander (2001),Zoolander 2 (2016),xXx (2002)
0,1,-8.326355e-03,-8.110858e-02,-1.021166e-02,-9.012270e-02,3.246242e-04,-4.768306e-03,1.014891e-01,-9.849866e-02,-7.596068e-02,...,2.142297e-03,1.733268e-17,2.885549e-04,-3.708682e-03,-1.589435e-03,9.055263e-03,7.922764e-02,-1.285615e-03,-3.887993e-03,4.173162e-03
1,2,1.039256e+01,2.558558e-02,5.124841e-04,-5.659740e-02,1.407735e-04,3.639586e-04,-3.974307e-02,3.593072e-02,3.507458e-02,...,-8.486194e-04,-2.610582e-16,1.251320e-04,2.830789e-04,1.213195e-04,-3.398369e-02,5.258358e-03,-1.740651e-02,1.437309e-01,-8.977774e-03
2,3,-1.637022e-02,1.458952e-01,-1.931053e-02,1.070135e-02,3.814509e-04,-2.281580e-03,-7.438640e-02,9.518509e-02,-2.119558e-02,...,1.058839e-05,3.434144e-18,3.390675e-04,-1.774562e-03,-7.605266e-04,7.060074e-02,-1.189207e-02,3.543378e-03,2.552496e-03,4.666808e-02
3,4,-4.506112e-02,-1.416855e-02,-3.543361e-02,2.160812e-01,-3.698316e-05,8.563210e-04,8.534996e-02,-4.519787e-02,2.880248e-01,...,-9.129552e-06,1.171165e-17,-3.287392e-05,6.660275e-04,2.854403e-04,3.236381e-02,-3.759160e-03,-1.003865e-02,1.600233e-03,2.688620e-02
4,5,2.774719e-16,1.811097e-16,-3.232256e-17,-5.660500e-17,-6.058540e-16,-8.245116e-17,2.696284e-16,5.036609e-17,6.779830e-17,...,3.165489e-18,9.640163e-34,-5.385369e-16,-6.412868e-17,-2.748372e-17,-1.762828e-16,1.250836e-16,8.994341e-17,-2.704683e-18,-1.127678e-16


In [22]:
def top_movie_recomender(user_ID):
    
    user_id=user_ID - 1
    
    orig_ratings=pd.DataFrame(df_new100_R.iloc[user_id].sort_values(ascending = False)).reset_index()
    orig_ratings.columns = ["movie","original_ratings"]
    final1=pd.merge(orig_ratings,df_mr[["movie","genre"]],how= "inner",on="movie").drop_duplicates()

    pred_ratings=pd.DataFrame(df_svd_new.iloc[user_id][1:].sort_values(ascending = False)).reset_index()
    pred_ratings.columns = ["movie","predicted_ratings"]
    final2=pd.merge(pred_ratings,df_mr[["movie","genre"]],how= "inner",on="movie").drop_duplicates()
    
    return final1,final2

In [23]:
df_orig,df_pred=top_movie_recomender(2)

In [32]:
df_orig.head(10)

,movie,original_ratings,genre
0,12 Years a Slave (2013),10.0,Biography|Drama|History
1809,Her (2013),10.0,Drama|Romance|Sci-Fi
3482,The Longest Ride (2015),9.0,Drama|Romance
3651,World War Z (2013),8.0,Action|Adventure|Horror
6038,Chef (2014),8.0,Comedy|Drama
6852,Spy (2015),8.0,Action|Comedy|Crime
7524,Straight Outta Compton (2015),8.0,Biography|Drama|History
8174,Disconnect (2012),8.0,Drama|Thriller
8692,Life of Pi (2012),8.0,Adventure|Drama|Fantasy
9629,In the Heart of the Sea (2015),8.0,Action|Adventure|Biography


In [33]:
df_pred.head(20)

,movie,predicted_ratings,genre
0,12 Years a Slave (2013),10.392560,Biography|Drama|History
1809,Her (2013),9.756232,Drama|Romance|Sci-Fi
3482,The Longest Ride (2015),8.780281,Drama|Romance
3651,Frozen (2013),8.454813,Animation|Adventure|Comedy
4860,Gravity (2013),8.352375,Sci-Fi|Thriller
7910,Straight Outta Compton (2015),8.137984,Biography|Drama|History
8560,Disconnect (2012),8.041212,Drama|Thriller
9078,World War Z (2013),7.890552,Action|Adventure|Horror
11465,Spy (2015),7.850791,Action|Comedy|Crime
12137,Life of Pi (2012),7.839769,Adventure|Drama|Fantasy
